# Lensing Kappa Maps

NOTICE HARDWIRED LINE 

### Basics

In [1]:
import numpy as np
import healpy as hp
import requests
import matplotlib as plt
from temdriver import rotate_map
import os

In [2]:
pathinputs = '/global/u1/m/manera/myKP3/kp3sys-inputs/'
pathout = '/global/u1/m/manera/myKP3/kp3sys-outputs/'
#pathinputs = '/Users/manera/desi_kp3/kp3_external_inputs/'

### Documentation

In [3]:
# ------ From Planck ----- 
# wiki https://wiki.cosmos.esa.int/planck-legacy-archive/index.php/Lensing 
# data archive: http://pla.esac.esa.int/pla/#home 
# package COM_Lensing_4096_R3.00 (the archive-web shows how to wget download from it) 
# lensing file: http://pla.esac.esa.int/pla/aio/product-action?COSMOLOGY.COSMOLOGY_OID=131&COSMOLOGY.FILE_ID=MV.tgz 
# mask file: http://pla.esac.esa.int/pla/aio/product-action? COSMOLOGY.COSMOLOGY_OID=130&COSMOLOGY.FILE_ID=mask.fits.gz 
# Also might be useful:
# Quijan maps https://gitlab.com/qianjunhang/desi-legacy-survey-superstructure-stacking/-/tree/master/data/Planck18
# Downloading with python https://www.codementor.io/@aviaryan/downloading-files-from-urls-in-python-77q3bs0un

### Download files

In [5]:
#Obtain maps 
url_lensalms = "http://pla.esac.esa.int/pla/aio/product-action?COSMOLOGY.COSMOLOGY_OID=131&COSMOLOGY.FILE_ID=MV.tgz" 
url_maskmap = "http://pla.esac.esa.int/pla/aio/product-action?COSMOLOGY.COSMOLOGY_OID=130&COSMOLOGY.FILE_ID=mask.fits.gz"

file_lensalms_tgz = "MV.tgz"
file_lensalms = "MV/dat_klm.fits"
file_mask = "mask.fits.gz" 

!wget -O {pathinputs+file_lensalms_tgz} "{url_lensalms}"
#!tar -xvf {pathinputs+file_lensalms_tgz} -C {pathinputs} "{file_lensalms}"
# HARDWIRED NEED TO UNTAR BY HAND!!! previous tar line did not work in nersc 
!wget -O {pathinputs+file_mask} "{url_maskmap}"

--2021-11-22 11:00:38--  http://pla.esac.esa.int/pla/aio/product-action?COSMOLOGY.COSMOLOGY_OID=131&COSMOLOGY.FILE_ID=MV.tgz
Resolving pla.esac.esa.int (pla.esac.esa.int)... 193.147.153.153
Connecting to pla.esac.esa.int (pla.esac.esa.int)|193.147.153.153|:80... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/x-binary]
Saving to: ‘/global/u1/m/manera/myKP3/kp3sys-inputs/MV.tgz’

/global/u1/m/manera     [   <=>              ] 306.66M  9.15MB/s    in 35s     

2021-11-22 11:01:14 (8.72 MB/s) - ‘/global/u1/m/manera/myKP3/kp3sys-inputs/MV.tgz’ saved [321559428]

--2021-11-22 11:01:14--  http://pla.esac.esa.int/pla/aio/product-action?COSMOLOGY.COSMOLOGY_OID=130&COSMOLOGY.FILE_ID=mask.fits.gz
Resolving pla.esac.esa.int (pla.esac.esa.int)... 193.147.153.153
Connecting to pla.esac.esa.int (pla.esac.esa.int)|193.147.153.153|:80... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/x-gzip]
Saving to: ‘/global/u1/m/man

option used here

another option

In [6]:
dd

note: to test map rotations one might use 
hp.projplot(theta,phi, 'bo') to plot a blue dot on coordinate (theta, phi)

### Function to create map from alms range 

In [6]:
def get_map_from_alms(alms,nside_map,nside_hires,ellmin,ellmax):
    # create map fom healpy alms 
    # 1) transform alm to map, with nside_res, only with ell <= ellmax
    # 2) if nside_map < nside_hires, degrades de map to nside_map 
    
    r = []
    for i in range(len(alms)):   
        if i < ellmin: 
            r.append(0.0)
        elif i > ellmax:        
            r.append(0.0)
        else:
            r.append(1.0)
    alm2=hp.almxfl(alms, r) #multiply kap_al by r 
    ptest = hp.alm2map(alm2,nside_hires)
    if(nside_map < nside_hires): 
        ptest = hp.ud_grade(ptest,nside_map)
    return ptest


### Create Lensing kappa Map 
- read alms file
- cut ells
- convert to map nside=4096
- rotate to equatorial
- degrade to desired nside 
- mollview plot
- save 

### Create Lensing mask Map 
- read map nside=2048
- rotate equatorial
- degrade 
- mollview plot
- save 


create maps nside = 256

In [ ]:
file_lensmap = 'lensmap_nside256.fits'
file_lensmask = 'lensmask_nside256.fits'

nside    = 256   # output resolution
nside_hi = 2048   # high resolution  
ellmin = 3        # remove monopole 0, dipole 1, quadrupole 2
ellmax = 2048     # remove very small structure
coordrot = coord = (['G','C']) 

# lensmap
fileinput = pathinputs+file_lensalms
alms = hp.read_alm(fileinput) 
lensmap = get_map_from_alms(alms,nside_hi,nside_hi,ellmin,ellmax)
lensmap = rotate_map(lensmap,coordrot)
lensmap = hp.ud_grade(lensmap,nside)

filename = pathout + file_lensmap
hp.mollview(lensmap,title='Lensing kappa',coord=["C"])
hp.write_map(filename, lensmap, nest=False) #function write empty healpix header 


# lensmask
fileinput = pathinputs+file_mask
lensmask = hp.read_map(fileinput)
lensmask = rotate_map(lensmask,coordrot)
lensmask = hp.ud_grade(lensmask,nside)

filename = pathout + file_lensmask
hp.mollview(lensmask,title='Lensing kappa',coord=["C"])
hp.write_map(filename, lensmask, nest=False) #function write empty healpix header 


create maps nside = 512

In [ ]:
file_lensmap = 'lensmap_nside512.fits'
file_lensmask = 'lensmask_nside512.fits'

nside    = 512   # output resolution
nside_hi = 2048   # high resolution  
ellmin = 3        # remove monopole 0, dipole 1, quadrupole 2
ellmax = 2048     # remove very small structure
coordrot = coord = (['G','C']) 

# lensmap
fileinput = pathinputs+file_lensalms
alms = hp.read_alm(fileinput) 
lensmap = get_map_from_alms(alms,nside_hi,nside_hi,ellmin,ellmax)
lensmap = rotate_map(lensmap,coordrot)
lensmap = hp.ud_grade(lensmap,nside)

filename = pathout + file_lensmap
hp.mollview(lensmap,title='Lensing kappa',coord=["C"])
hp.write_map(filename, lensmap, nest=False) #function write empty healpix header 


# lensmask
fileinput = pathinputs+file_mask
lensmask = hp.read_map(fileinput)
lensmask = rotate_map(lensmask,coordrot)
lensmask = hp.ud_grade(lensmask,nside)

filename = pathout + file_lensmask
hp.mollview(lensmask,title='Lensing kappa',coord=["C"])
hp.write_map(filename, lensmask, nest=False) #function write empty healpix header 


### ----- ------ ------- ------- ------- -------

In [1]:
## Extra, not-needed
## If I wanted some maps from Quian Junhang
#url = 'https://gitlab.com/qianjunhang/desi-legacy-survey-superstructure-stacking/-/raw/master/data/Planck18/planck2018-ellmax2048_lensmap_512_equatorial_alm.fits?inline=false'
#pathsave='/global/u1/m/manera/myKP3/kp3sys-inputs/'
#
## the correct filename can't be get now with url.rspli
#filename = 'planck2018-ellmax2048_lensmap_512_equatorial_alms.fits' 
#print (filename)
#
#r = requests.get(url, allow_redirects=True)
#open(pathsave+filename, 'wb').write(r.content)
